In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import os

app = FastAPI() # 创建 api 对象


In [ ]:
# 定义一个数据模型，用于接收POST请求中的数据
class Item(BaseModel):
    prompt : str # 用户 prompt
    model : str = "ERNIE-Bot-4"# 使用的模型
    temperature : float = 0.1# 温度系数
    if_history : bool = False # 是否使用历史对话功能
    # API_Key
    api_key: str = None
    # Secret_Key
    secret_key : str = None
    # access_token
    access_token: str = None
    # APPID
    appid : str = None
    # APISecret
    Spark_api_secret : str = None
    # Secret_key
    Wenxin_secret_key : str = None
    # 数据库路径
    db_path : str = "../database/vector_wenxin_db"
    # 源文件路径
    file_path : str = "../../data_base/knowledge_db"
    # prompt template
    prompt_template : str = template
    # Template 变量
    input_variables : list = ["context","question"]
    # Embdding
    embedding : str = "wenxin"
    # Top K
    top_k : int = 5
    # embedding_key
    embedding_key : str = None

In [ ]:
@app.post("/answer/")
async def get_response(item: Item):
    # 首先确定需要调用的链
    if not item.if_history:
        # 调用 Chat 链
        # return item.embedding_key
        if item.embedding_key == None:
            # wenxin比较特殊，要传入ak和sk鉴权得到access_token
            if item.embedding == "wenxin":
                item.embedding_key = [item.api_key, item.Wenxin_secret_key]
            # 否则call_embedding.py内的parse_llm_api_key会读取环境变量进行赋值
        chain = QA_chain_self(model=item.model, temperature=item.temperature, 
                              top_k=item.top_k, file_path=item.file_path, 
                              persist_path=item.db_path, 
                              appid=item.appid, api_key=item.api_key, 
                              embedding=item.embedding, template=template, 
                              Spark_api_secret=item.Spark_api_secret, 
                              Wenxin_secret_key=item.Wenxin_secret_key, 
                              embedding_key=item.embedding_key)

        response = chain.answer(question = item.prompt)
    
        return response
    
    # 由于 API 存在即时性问题，不能支持历史链
    else:
        return "API 不支持历史链"

In [11]:
# 加载.env文件
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
# _ = load_dotenv(find_dotenv())
_ = load_dotenv('../QA_Project/.env')

# 获取环境变量 
wenxin_api_key = os.environ["wenxin_api_key"]
wenxin_secret_key = os.environ["wenxin_secret_key"]

In [12]:
import requests

url = "http://127.0.0.1:8000/answer"

data = {
    "prompt":"什么是蘑菇书？",
    "api_key":wenxin_api_key,
    "Wenxin_secret_key":wenxin_secret_key,
}

r = requests.post(url, json=data, headers = {"Content-Type": "application/json"})

In [13]:
print(r)

<Response [200]>


In [14]:
r.text

'"蘑菇书是一本介绍强化学习知识的书籍，全书共分为十三章，涵盖了强化学习的基础知识和传统算法，以及适用强化学习的算法和常见问题的解决方法。书名寓意来源于超级玛里奥游戏中的蘑菇，象征着读者在阅读本书后，可以像玛里奥一样变得更加强大。谢谢你的提问！"'